In [1]:
import pandas as pd
import seaborn as sns
import re
import numpy as np

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train_test = [train,test] #train 과 test data 합침

In [3]:
def bar_chart(feature):
    survived = train[train['Survived']==1][feature].value_counts()
    dead = train[train['Survived']==0][feature].value_counts()
    df=pd.DataFrame([survived, dead])
    df.index= ['Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5))

In [4]:
def get_title(name):
    match = re.search(' ([A-Za-z]+)\.', name)
    if match:
        return match.group()
    else:
        return NaN

In [5]:
train.describe()
#Age의 min, max, mean 값 확인

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
# Before Data preprocessing
train.head(5) #: 5에 NaN
#train.isnull().sum()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
test.head(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [8]:
sex_map = {"male":0,"female":1}
embarked_map = {'S':0,'Q':1,'C':2}
boolean_map = {False:0,True:1}
title_map = {" Mr.":0, " Miss.":1," Mrs.":2," Master.":3,
             " Dr.":4," Don.":4, " Rev.":4," Major.":4, " Mlle.":1, " Col.":4, " Countess.":4, " Mme.":2,
            " Ms.":1, " Lady.":4," Jonkheer.":4," Don.":4," Dona.":4," Capt.":4," Sir.":4}

train['Sex'] = train['Sex'].map(sex_map)

train['Embarked'] = train['Embarked'].map(embarked_map)
train['Embarked'] = train['Embarked'].fillna(0)#누락된값이 2개밖에 없어서 그냥 S로 했습니다.

train['Cabin'] = train['Cabin'].notnull()
train['Cabin'] = train['Cabin'].map(boolean_map)

train['Title'] = train['Name'].map(get_title)
train['Title'] = train['Title'].map(title_map)
train['Title'] = train['Title'].fillna(0)

#pd.crosstab(train["Title"],train["Sex"]) #추출한 Title 과 성별 같이 표현


In [9]:
#Age의 NaN에 각 그룹의 평균값 대입
train["Age"].fillna(train.groupby("Title")["Age"].transform("median"),inplace=True)

#부모,자식수(Parch) + 형제자매,배우자(SibSp) = 동행자 수
train['FamilySize'] = train['SibSp']+train['Parch']

In [10]:
test['Sex'] = test['Sex'].map(sex_map)

test['Embarked'] = test['Embarked'].map(embarked_map)
test['Embarked'] = test['Embarked'].fillna(0)

test['Cabin'] = test['Cabin'].notnull()
test['Cabin'] = test['Cabin'].map(boolean_map)#Cabin의 경우에 기록 유무에만 따라 0 1 로 했어요

test['Title'] = test['Name'].map(get_title)
test['Title'] = test['Title'].map(title_map)
test['Title'] = test['Title'].fillna(0.3)#이것두 Title의 조정 수치

#Age의 NaN에 각 그룹의 평균값 대입
test["Age"].fillna(test.groupby("Title")["Age"].transform("median"),inplace=True)

test['FamilySize'] = test['SibSp']+test['Parch']

In [11]:
#train.drop('PassengerId',axis=1,inplace=True)
train.drop('Name',axis=1,inplace=True)
train.drop('Ticket',axis=1,inplace=True)
train.drop('Fare',axis=1,inplace=True)
train.drop('SibSp',axis=1,inplace=True)
train.drop('Parch',axis=1,inplace=True)

test.drop('Name',axis=1,inplace=True)
test.drop('Ticket',axis=1,inplace=True)
test.drop('Fare',axis=1,inplace=True)
test.drop('SibSp',axis=1,inplace=True)
test.drop('Parch',axis=1,inplace=True)

test_data=test.drop('PassengerId',axis=1).copy()

In [13]:
for data in train_test:
    data.loc[ data["Age"] <= 15, "Age"] = 0,
    data.loc[ (data["Age"] > 15) & (data["Age"] <= 30), "Age"] = 1,
    data.loc[ (data["Age"] > 30) & (data["Age"] <= 45), "Age"] = 2,
    data.loc[ (data["Age"] > 45) & (data["Age"] <= 60), "Age"] = 3,
    data.loc[ data["Age"] > 60, "Age"] = 4
    
#Data Binning

In [14]:
#train.isnull().sum()
#train.info()
#bar_chart("Embarked")

#After Data preprocessing

#train.head(10)

In [15]:
#bar_chart("FamilySize")
train.head(5)
#train.shape

,PassengerId,Survived,Pclass,Sex,Age,Cabin,Embarked,Title,FamilySize
0,1,0,3,0,1.0,0,0.0,0,1
1,2,1,1,1,2.0,1,2.0,2,1
2,3,1,3,1,1.0,0,0.0,1,0
3,4,1,1,1,2.0,1,0.0,2,1
4,5,0,3,0,2.0,0,0.0,0,0


In [16]:
#train.isnull().sum()
test.head(5)
#test.shape

,PassengerId,Pclass,Sex,Age,Cabin,Embarked,Title,FamilySize
0,892,3,0,2.0,0,1,0,0
1,893,3,1,3.0,0,0,2,1
2,894,2,0,4.0,0,1,0,0
3,895,3,0,1.0,0,0,0,0
4,896,3,1,1.0,0,0,2,2


In [17]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from tensorflow.contrib import learn

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.utils import shuffle

import numpy as np

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [18]:
#train.info()

train_data = train.drop('Survived', axis =1)
train_data.drop('PassengerId',axis=1,inplace=True)

target = train['Survived']
#train_data, target = shuffle(train_data, target, random_state =5)
train_data.shape, target.shape

((891, 7), (891,))

In [19]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [20]:
clf = KNeighborsClassifier(n_neighbors = 13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv = k_fold, n_jobs=1, scoring = scoring)
print(score)

[0.82222222 0.80898876 0.80898876 0.78651685 0.82022472 0.79775281
 0.82022472 0.79775281 0.82022472 0.82022472]


In [21]:
round(np.mean(score)*100,2) #kNN =81.71

81.03

In [22]:
clf = DecisionTreeClassifier() #의사결정트리

scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv=k_fold, n_jobs=1, scoring = scoring)
print(score)

[0.82222222 0.80898876 0.80898876 0.78651685 0.83146067 0.7752809
 0.82022472 0.79775281 0.71910112 0.79775281]


In [23]:
round(np.mean(score)*100, 2) #DT = 79.58

79.68

In [24]:
clf = RandomForestClassifier(n_estimators=13)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv = k_fold, n_jobs=1, scoring = scoring)
print(score)

[0.83333333 0.83146067 0.79775281 0.80898876 0.84269663 0.78651685
 0.80898876 0.79775281 0.74157303 0.79775281]


In [25]:
round(np.mean(score)*100, 2) #RF 80.03%

80.47

In [26]:
clf = GaussianNB()
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv = k_fold, n_jobs=1, scoring = scoring)
print(score)

[0.85555556 0.75280899 0.75280899 0.74157303 0.73033708 0.7752809
 0.76404494 0.80898876 0.86516854 0.84269663]


In [27]:
round(np.mean(score)*100, 2) #NB=79.35

78.89

In [28]:
clf = SVC(C=1, kernel='rbf', coef0=1)
scoring = 'accuracy'
score = cross_val_score(clf, train_data, target, cv = k_fold, n_jobs=1, scoring = scoring)
print(score)

[0.82222222 0.79775281 0.83146067 0.82022472 0.84269663 0.80898876
 0.82022472 0.82022472 0.82022472 0.87640449]


In [29]:
round(np.mean(score)*100, 2) #SVM = 82.72

82.6

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
clf = LogisticRegression()
clf.fit(train_data, target)
prediction = clf.predict(test_data)
round(clf.score(train_data, target)*100,2) #LR : 81.82

81.82

In [32]:
#train.shape, test.shape, train_data.shape, test_data.shape

In [33]:
#SVM이 82.72으로 가장 높다!

#clf = SVC(C=1, kernel ='rbf', coef0=1)
#clf = KNeighborsClassifier(n_neighbors = 13)
#clf = RandomForestClassifier(n_estimators=100)
#clf = DecisionTreeClassifier()
clf = GaussianNB()
clf.fit(train_data, target)

GaussianNB(priors=None)

In [34]:

prediction = clf.predict(test_data)

In [35]:
import collections, numpy
collections.Counter(prediction)

Counter({1: 410, 0: 8})

In [36]:
submission = pd.DataFrame({
    "PassengerId" : test["PassengerId"],
    "Survived" : prediction
})

submission.to_csv('submission.csv', index = False)

In [37]:
submission = pd.read_csv('submission.csv')
submission.head(5)

,PassengerId,Survived
0,892,1
1,893,1
2,894,1
3,895,1
4,896,1
